<a href="https://colab.research.google.com/github/lightwsrld/pytorch/blob/main/transformer_model_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchtext==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 989.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.2
    Uninstalling torchtext-0.15.2:
      Successfully uninstalled torchtext-0.15.2


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os,gc,re,time
import matplotlib.pyplot as plt
import seaborn as sns
import spacy,random
from collections import Counter
import warnings,unicodedata
warnings.simplefilter("ignore")


#setting device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Available device: {device}')

Available device: cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

In [ ]:
train_csv_path, val_csv_path, test_csv_path = 'train_1.csv','val_1.csv','test_1.csv'
train_data = pd.read_csv(train_csv_path)
val_data = pd.read_csv(val_csv_path)
test_data = pd.read_csv(test_csv_path)

print(train_data.shape, val_data.shape, test_data.shape)

(29000, 2) (1014, 2) (1000, 2)


In [ ]:
import re,unicodedata
import time,datetime

In [ ]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

2023-08-08 02:27:39.397869: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 80.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
2023-08-08 02:27:54.680565: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 83.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
def unicodeToAscii(s):

    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

replace_dict = {"don't": "do not","shouldn't": "should not",
         "can't": "cannot", "I'm": "Iam", "I've": "I have",
         "haven't": "have not",
         "hasn't": "has not",
         "didn't": "did not"}

def clean_eng_text(text, replace_dict=replace_dict):

    text = unicodeToAscii(text.lower().strip())
    text = re.sub('[-?<>.!]', '', text)
    text = re.sub('\s+', ' ', text)
    text = ' '.join([replace_dict[word] if word in replace_dict.keys() else word for word in text.split()])
    text = [re.sub('[^A-Za-z]', '', word) for word in text.split()]
    return ' '.join(text).lower()

def clean_ger_text(text):

    text = text.strip()
    text = re.sub("[.?]", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    return text

In [ ]:
train_data["English"] = train_data["English"].apply(lambda x: clean_eng_text(x))
train_data["Ger"] = train_data["Ger"].apply(lambda x: clean_ger_text(x))

val_data["English"] = val_data["English"].apply(lambda x: clean_eng_text(x))
val_data["Ger"] = val_data["Ger"].apply(lambda x: clean_ger_text(x))

test_data["English"] = test_data["English"].apply(lambda x: clean_eng_text(x))
test_data["Ger"] = test_data["Ger"].apply(lambda x: clean_ger_text(x))

In [ ]:
import en_core_web_sm, de_core_news_sm

# 단어 사전 형성
class VocabGenerator:
    def __init__(self, corpus, min_frequency=2, tokenizer_lang="english"):

        self.stoi = {'<UNK>':1,'<PAD>':0,'<SOS>':2,'<EOS>':3}          # 문자열을 index 변환
        self.itos = {1:'<UNK>',0:'<PAD>',2:'<SOS>',3:'<EOS>'}          # index를 문자열로 변환
        self.min_freq = min_frequency

        # 영어, 독일어 문장을 각각 개별 토큰으로 변환
        self.lang = tokenizer_lang
        if tokenizer_lang == "english":
            self.tokenizer = en_core_web_sm.load()
        else:
            self.tokenizer = de_core_news_sm.load()

        self.build_vocabulary(corpus)

    def __len__(self):

        return len(self.itos)

    def counter(self,text):

        out = {}
        text = text.split()
        for word in text:
            if word in out:
                out[word] += 1
            else:
                out[word] = 1
        return out

    def build_vocabulary(self,corpus):

        idx = 4
        frequency_dict = self.counter(corpus)
        for word,freq in frequency_dict.items():
          if freq >= self.min_freq:
              self.stoi[word] = idx
              self.itos[idx] = word
              idx += 1

    def generate_numeric_tokens(self,text):

        tokenized_out = [token.text for token in self.tokenizer(text)]
        out = [self.stoi[token] if token in self.stoi.keys() else self.stoi["<UNK>"] for token in tokenized_out]
        return out

    def add_eos_sos(self,numeric_list):

        numeric_list = [self.stoi['<SOS>']] + numeric_list + [self.stoi['<EOS>']]      # 토큰에서 생성된 정수 목록
        return numeric_list

    def pad_sequence(self,numeric_list,max_seq_length):
      # 토큰의 정수 목록을 패딩하고 잘라내는 함

        if len(numeric_list) < max_seq_length:
            no_zeros = max_seq_length - len(numeric_list)
            numeric_list = numeric_list + [self.stoi['<PAD>'] for i in range(no_zeros)]
        else:
            numeric_list = numeric_list[:max_seq_length]

        return numeric_list

In [ ]:
from torch.utils.data import Dataset, DataLoader
max_seq_len=50

class TranslationDataset(Dataset):
    def __init__(self, data, src_vocab, trg_vocab):
        self.data = data
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab

    def __len__(self):
        return len(self.data)

    def preprocess(self, text, lang):
        tokenized_out = [token.text for token in lang(text)]
        out = [self.src_vocab.stoi[token] if token in self.src_vocab.stoi.keys() else self.src_vocab.stoi[""] for token in tokenized_out]
        return out

    def add_eos_sos(self, numeric_list):
        numeric_list = [self.src_vocab.stoi['']] + numeric_list + [self.src_vocab.stoi['']]
        return numeric_list

    def pad_sequence(self, numeric_list, max_seq_length):
        if len(numeric_list) < max_seq_length:
            no_zeros = max_seq_length - len(numeric_list)
            numeric_list = numeric_list + [self.src_vocab.stoi[''] for i in range(no_zeros)]
        else:
            numeric_list = numeric_list[:max_seq_length]
        return numeric_list

    def __getitem__(self, idx):

        src_text = self.data['src_column_name'].iloc[idx]
        trg_text = self.data['trg_column_name'].iloc[idx]

        src_tokens = self.preprocess(src_text, self.src_vocab)
        trg_tokens = self.preprocess(trg_text, self.trg_vocab)
        src_tokens = self.add_eos_sos(src_tokens)[:max_seq_len-2]
        trg_tokens = self.add_eos_sos(trg_tokens)[:max_seq_len-2]

        src_tokens = self.pad_sequence(src_tokens, max_seq_len)
        trg_tokens = self.pad_sequence(trg_tokens, max_seq_len)

        return torch.tensor(src_tokens), torch.tensor(trg_tokens)

In [ ]:
corpus_eng = ' '.join(list(train_data['English'].values))
TRG = VocabGenerator(corpus=corpus_eng, min_frequency=2,tokenizer_lang="english")

corpus_ger = ' '.join(list(train_data['Ger'].values))
SRC = VocabGenerator(corpus=corpus_ger, min_frequency=2,tokenizer_lang="germen")

In [ ]:
print(f"len(SRC): {len(SRC)}")
print(f"len(TRG): {len(TRG)}")

len(SRC): 7971
len(TRG): 5918


In [ ]:
print(TRG.stoi["<PAD>"])  # 패딩(padding): 1
print(TRG.stoi["<SOS>"])        #  : 2
print(TRG.stoi["<EOS>"])       # : 3
print(TRG.stoi["water"])
print(TRG.stoi["world"])

0
2
3
264
4070


In [ ]:
tokenizers = {
    SRC: lambda x: [SRC.stoi[tok] for tok in SRC.generate_numeric_tokens(x)],
    TRG: lambda x: [TRG.stoi[tok] for tok in TRG.generate_numeric_tokens(x)]
}

BOS, EOS, PAD = 2, 3, 0

In [ ]:
from torch.utils.data import Dataset, DataLoader
max_seq_len=50

class TranslationDataset(Dataset):
    'create a dataset for torch.utils.data.DataLoader() '
    def __init__(self, data, src_vocab, trg_vocab):
        self.data = data
        self.src_vocab = src_vocab  # Change SRC to src_vocab
        self.trg_vocab = trg_vocab  # Change TRG to trg_vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_text, trg_text = self.data.iloc[idx]

        # 토큰화하고 문장의 처음과 끝에 각각 BOS, EOS를 추
        src_tokens = [BOS] + self.src_vocab.generate_numeric_tokens(src_text)[:max_seq_len-2] + [EOS]
        trg_tokens = [BOS] + self.trg_vocab.generate_numeric_tokens(trg_text)[:max_seq_len-2] + [EOS]

        return torch.tensor(src_tokens), torch.tensor(trg_tokens)

In [ ]:
# 배치의 모든 문장이 동일한 길이를 갖도록 문장에 패딩을 추가하는 함수
def pad_sequence(batch):

    src_seqs = [src for src, trg in batch]
    trg_seqs = [trg for src, trg in batch]
    src_padded = torch.nn.utils.rnn.pad_sequence(src_seqs,
                                batch_first=True, padding_value=PAD)
    trg_padded = torch.nn.utils.rnn.pad_sequence(trg_seqs,
                                batch_first=True, padding_value=PAD)
    return src_padded, trg_padded

batch_size = 128

In [ ]:
train_data = TranslationDataset(train_data, SRC, TRG)
valid_data = TranslationDataset(val_data, SRC, TRG)
test_data = TranslationDataset(test_data, SRC, TRG)

In [ ]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=pad_sequence)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True, collate_fn=pad_sequence)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, collate_fn=pad_sequence)

In [ ]:
import math

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_embed, dropout=0.0):
        super(MultiHeadedAttention, self).__init__()
        assert d_embed % h == 0
        self.d_k = d_embed//h      # dimension of each attention head
        self.d_embed = d_embed     # embedding dimension
        self.h = h                 # numbers of attention head
        self.WQ = nn.Linear(d_embed, d_embed)
        self.WK = nn.Linear(d_embed, d_embed)
        self.WV = nn.Linear(d_embed, d_embed)
        self.linear = nn.Linear(d_embed, d_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x_query, x_key, x_value, mask=None):
        nbatch = x_query.size(0)           # batch size

        # 1) Linear projections to get the multi-head query, key and value tensors
        # x_query, x_key, x_value dimension: nbatch * seq_len * d_embed

        query = self.WQ(x_query).view(nbatch, -1, self.h, self.d_k).transpose(1,2)
        key   = self.WK(x_key).view(nbatch, -1, self.h, self.d_k).transpose(1,2)
        value = self.WV(x_value).view(nbatch, -1, self.h, self.d_k).transpose(1,2)

        # 2) Self-attention
        # scores dimensions (sequence length): nbatch * h * seq_len * seq_len
        scores = torch.matmul(query, key.transpose(-2, -1))/math.sqrt(self.d_k)

        # 3) Mask out padding tokens and future tokens
        if mask is not None:
            scores = scores.masked_fill(mask, float('-inf'))
        # p_atten dimensions: nbatch * h * seq_len * seq_len
        p_atten = torch.nn.functional.softmax(scores, dim=-1)
        p_atten = self.dropout(p_atten)
        # x dimensions: nbatch * h * seq_len * d_k
        x = torch.matmul(p_atten, value)
        # x dimensions: nbtach * seq_len * d_embed
        x = x.transpose(1, 2).contiguous().view(nbatch, -1, self.d_embed)
        return self.linear(x)              # final linear layer

class ResidualConnection(nn.Module):
  '''residual connection: x + dropout(sublayer(layernorm(x))) '''
  def __init__(self, dim, dropout):
      super().__init__()
      self.drop = nn.Dropout(dropout)
      self.norm = nn.LayerNorm(dim)

  def forward(self, x, sublayer):
      return x + self.drop(sublayer(self.norm(x)))

class Encoder(nn.Module):
    '''Encoder = token embedding + positional embedding -> a stack of N EncoderBlock -> layer norm'''
    def __init__(self, config):
        super().__init__()
        self.d_embed = config.d_embed
        self.tok_embed = nn.Embedding(config.encoder_vocab_size, config.d_embed)
        self.pos_embed = nn.Parameter(torch.zeros(1, config.max_seq_len, config.d_embed))
        self.encoder_blocks = nn.ModuleList([EncoderBlock(config) for _ in range(config.N_encoder)])
        self.dropout = nn.Dropout(config.dropout)
        self.norm = nn.LayerNorm(config.d_embed)

    def forward(self, input, mask=None):
        x = self.tok_embed(input)
        x_pos = self.pos_embed[:, :x.size(1), :]
        x = self.dropout(x + x_pos)
        for layer in self.encoder_blocks:
            x = layer(x, mask)
        return self.norm(x)

class EncoderBlock(nn.Module):
    '''EncoderBlock: self-attention -> position-wise fully connected feed-forward layer'''
    def __init__(self, config):
        super(EncoderBlock, self).__init__()
        self.atten = MultiHeadedAttention(config.h, config.d_embed, config.dropout)
        self.feed_forward = nn.Sequential(
            nn.Linear(config.d_embed, config.d_ff),
            nn.ReLU(),
            nn.Dropout(config.dropout),
            nn.Linear(config.d_ff, config.d_embed)
        )
        self.residual1 = ResidualConnection(config.d_embed, config.dropout)
        self.residual2 = ResidualConnection(config.d_embed, config.dropout)

    def forward(self, x, mask=None):
        x = self.residual1(x, lambda x: self.atten(x, x, x, mask=mask))
        # position-wise fully connected feed-forward layer
        return self.residual2(x, self.feed_forward)

class Decoder(nn.Module):
    '''Decoder = token embedding + positional embedding -> a stack of N DecoderBlock -> fully-connected layer'''
    def __init__(self, config):
        super().__init__()
        self.d_embed = config.d_embed
        self.tok_embed = nn.Embedding(config.decoder_vocab_size, config.d_embed)
        self.pos_embed = nn.Parameter(torch.zeros(1, config.max_seq_len, config.d_embed))
        self.dropout = nn.Dropout(config.dropout)
        self.decoder_blocks = nn.ModuleList([DecoderBlock(config) for _ in range(config.N_decoder)])
        self.norm = nn.LayerNorm(config.d_embed)
        self.linear = nn.Linear(config.d_embed, config.decoder_vocab_size)

    # Subsequent Masking
    def future_mask(self, seq_len):
        '''mask out tokens at future positions'''
        mask = (torch.triu(torch.ones(seq_len, seq_len, requires_grad=False), diagonal=1)!=0).to(DEVICE)
        return mask.view(1, 1, seq_len, seq_len)

    def forward(self, memory, src_mask, trg, trg_pad_mask):
        seq_len = trg.size(1)
        trg_mask = torch.logical_or(trg_pad_mask, self.future_mask(seq_len))
        x = self.tok_embed(trg) + self.pos_embed[:, :trg.size(1), :]
        x = self.dropout(x)
        for layer in self.decoder_blocks:
            x = layer(memory, src_mask, x, trg_mask)
        x = self.norm(x)
        logits = self.linear(x)
        return logits


class DecoderBlock(nn.Module):
    ''' EncoderBlock: self-attention -> position-wise feed-forward (fully connected) layer'''
    def __init__(self, config):
        super().__init__()
        self.atten1 = MultiHeadedAttention(config.h, config.d_embed)
        self.atten2 = MultiHeadedAttention(config.h, config.d_embed)
        self.feed_forward = nn.Sequential(
            nn.Linear(config.d_embed, config.d_ff),
            nn.ReLU(),
            nn.Dropout(config.dropout),
            nn.Linear(config.d_ff, config.d_embed)
        )
        self.residuals = nn.ModuleList([ResidualConnection(config.d_embed, config.dropout)
                                       for i in range(3)])

    def forward(self, memory, src_mask, decoder_layer_input, trg_mask):
        x = memory
        y = decoder_layer_input
        y = self.residuals[0](y, lambda y: self.atten1(y, y, y, mask=trg_mask))
        # keys and values are from the encoder output
        y = self.residuals[1](y, lambda y: self.atten2(y, x, x, mask=src_mask))
        return self.residuals[2](y, self.feed_forward)


class Transformer(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, src_mask, trg, trg_pad_mask):
        return self.decoder(self.encoder(src, src_mask), src_mask, trg, trg_pad_mask)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cpu


In [ ]:
class ModelConfig:
    def __init__(self, encoder_vocab_size, decoder_vocab_size, d_embed, d_ff, h,
                 N_encoder, N_decoder, max_seq_len, dropout):
        self.encoder_vocab_size = encoder_vocab_size
        self.decoder_vocab_size = decoder_vocab_size
        self.d_embed = d_embed
        self.d_ff = d_ff
        self.h = h
        self.N_encoder = N_encoder
        self.N_decoder = N_decoder
        self.max_seq_len = max_seq_len
        self.dropout = dropout

def make_model(config):
    model = Transformer(Encoder(config), Decoder(config)).to(DEVICE)

    for p in model.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    return model

In [ ]:
def make_batch_input(x, y):
        src = x.to(DEVICE)
        trg_in = y[:, :-1].to(DEVICE)
        trg_out = y[:, 1:].contiguous().view(-1).to(DEVICE)
        # padding tokens in the scr and trg sequences
        src_pad_mask = (src == PAD).view(src.size(0), 1, 1, src.size(-1))
        trg_pad_mask = (trg_in == PAD).view(trg_in.size(0), 1, 1, trg_in.size(-1))
        return src, trg_in, trg_out, src_pad_mask, trg_pad_mask

In [ ]:
en_vocab_size = 8200
de_vocab_size = 10000
vocab_sizes = {"TRG": en_vocab_size, "SRC": de_vocab_size}

In [ ]:
input_vocab_dim = SRC.__len__()
target_vocab_dim = TRG.__len__()

In [ ]:
config = ModelConfig(encoder_vocab_size = input_vocab_dim,
                     decoder_vocab_size=target_vocab_dim,
                     d_embed = 512,
                     d_ff = 512,
                     h = 8,
                     N_encoder = 3,
                     N_decoder = 3,
                     max_seq_len = max_seq_len,
                     dropout=0.1
                     )

train_size = len(train_loader) * batch_size
model = make_model(config)
model_size = sum([p.numel() for p in model.parameters()])
print(f'model_size: {model_size}, train_set_size: {train_size}')
warmup_steps = 3 * len(train_loader)

model_size: 22823198, train_set_size: 29056


In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

Transformer(
  (encoder): Encoder(
    (tok_embed): Embedding(7971, 512)
    (encoder_blocks): ModuleList(
      (0-2): 3 x EncoderBlock(
        (atten): MultiHeadedAttention(
          (WQ): Linear(in_features=512, out_features=512, bias=True)
          (WK): Linear(in_features=512, out_features=512, bias=True)
          (WV): Linear(in_features=512, out_features=512, bias=True)
          (linear): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): ReLU()
          (2): Dropout(p=0.1, inplace=False)
          (3): Linear(in_features=512, out_features=512, bias=True)
        )
        (residual1): ResidualConnection(
          (drop): Dropout(p=0.1, inplace=False)
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (residual2): ResidualConnection(
          (drop): Dro

In [ ]:
from numpy.lib.utils import lookfor
from tqdm import tqdm

def train_epoch(model, dataloader):
    model.train()
    grad_norm_clip = 1.0
    losses = []
    num_batches = len(dataloader)
    pbar = tqdm(enumerate(dataloader), total=num_batches)
    for idx, (x, y) in pbar:
        optimizer.zero_grad()
        src, trg_in, trg_out, src_pad_mask, trg_pad_mask = make_batch_input(x, y)
        pred = model(src, src_pad_mask, trg_in, trg_pad_mask).to(DEVICE)
        pred = pred.view(-1, pred.size(-1))
        loss = loss_fn(pred, trg_out).to(DEVICE)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_norm_clip)
        optimizer.step()
        scheduler.step()
        losses.append(loss.item())
        # report progress
        if idx > 0 and idx % 50 == 0:
            pbar.set_description(f'train loss={loss.item():.3f}, lr={scheduler.get_last_lr()[0]:.5f}')
    return np.mean(losses)


def train(model, train_loader, valid_loader, epochs):
    global early_stop_count
    best_valid_loss = float('inf')
    train_size = len(train_loader.dataset)
    pad_token_idx = TRG.stoi['<PAD>']
    loss_fn = nn.CrossEntropyLoss(ignore_index=pad_token_idx)
    for ep in range(epochs):
        train_loss = train_epoch(model, train_loader)
        valid_loss = validate(model, valid_loader)

        print(f'ep: {ep}: train_loss={train_loss:.5f}, valid_loss={valid_loss:.5f}')
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
        else:
            if scheduler.last_epoch > 2 * warmup_steps:
                early_stop_count -= 1
                if early_stop_count <= 0:
                    return train_loss, valid_loss
    return train_loss, valid_loss


def validate(model, dataloder):
    'compute the validation loss'
    model.eval()
    losses = []
    with torch.no_grad():
        for i, (x, y) in enumerate(dataloder):
            src, trg_in, trg_out, src_pad_mask, trg_pad_mask = make_batch_input(x,y)
            pred = model(src, src_pad_mask, trg_in, trg_pad_mask).to(DEVICE)
            pred = pred.view(-1, pred.size(-1))
            losses.append(loss_fn(pred, trg_out).item())
    return np.mean(losses)

In [ ]:
def translate(model, x):
    'translate source sentences into the target language, without looking at the answer'
    with torch.no_grad():
        dB = x.size(0)
        y = torch.tensor([[BOS]*dB]).view(dB, 1).to(DEVICE)
        x_pad_mask = (x == PAD).view(x.size(0), 1, 1, x.size(-1)).to(DEVICE)
        memory = model.encoder(x, x_pad_mask)
        for i in range(max_seq_len):
            y_pad_mask = (y == PAD).view(y.size(0), 1, 1, y.size(-1)).to(DEVICE)
            logits = model.decoder(memory, x_pad_mask, y, y_pad_mask)
            last_output = logits.argmax(-1)[:, -1]
            last_output = last_output.view(dB, 1)
            y = torch.cat((y, last_output), 1).to(DEVICE)
    return y

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 2.3 MB/s eta 0:00:00


In [ ]:
import sacrebleu

def remove_pad(sent):
    '''truncate the sentence if BOS is in it,
     otherwise simply remove the padding tokens at the end'''
    if sent.count('')>0:
      sent = sent[0:sent.index('')+1]
    while sent and sent[-1] == '':
            sent = sent[:-1]
    return sent

def decode_sentence(sentence_ids, vocab):
    'convert a list of numbers representing a tokenized sentence to a literal string'
    sentence = [vocab.itos[id] for id in sentence_ids]
    sentence = " ".join(sentence).replace("▁", "").strip().replace(" .", ".")
    return sentence

def evaluate(model, DataLoader, num_batch=None):
    'evaluate the model, and compute the BLEU score'
    model.eval()
    refs, cans, bleus = [], [], []
    with torch.no_grad():
        for idx, (x, y) in enumerate(DataLoader):
            src, trg_in, trg_out, src_pad_mask, trg_pad_mask = make_batch_input(x, y)
            translation = translate(model, src)
            # Convert trg_out and translation to lists
            refs = refs + [decode_sentence(trg_out[i].tolist(), TRG) for i in range(len(src))]
            cans = cans + [decode_sentence(translation[i].tolist(), TRG) for i in range(len(src))]
            if num_batch and idx >= num_batch:
                break
        print(min([len(x) for x in refs]))
        bleus.append(sacrebleu.corpus_bleu(cans, [refs]).score)

        # print some examples
        for i in range(3):
            print(f'src:  {decode_sentence(src[i].tolist(), SRC)}')
            print(f'trg:  {decode_sentence(trg_out[i].tolist(), TRG)}')
            print(f'pred: {decode_sentence(translation[i].tolist(), TRG)}')
        return np.mean(bleus)


In [ ]:
lr_fn = lambda step: config.d_embed**(-0.5) * min([(step + 1) ** (-0.5), (step + 1) * warmup_steps**(-1.5)])
optimizer = torch.optim.Adam(model.parameters(), lr=0.5, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_fn)
loss_fn = nn.CrossEntropyLoss(ignore_index=TRG.stoi['<PAD>'])
early_stop_count = 2
train_loss, valid_loss = train(model, train_loader, valid_loader, epochs=1)
test_loss = validate(model, test_loader)

print("train set examples:")
train_bleu = evaluate(model, train_loader, 20)
print("validation set examples:")
valid_bleu = evaluate(model, valid_loader)
print("test set examples:")
test_bleu  = evaluate(model, test_loader)
print(f'train_loss: {train_loss:.4f}, valid_loss: {valid_loss:.4f}, test_loss: {test_loss:.4f}')
print(f'test_bleu: {test_bleu:.4f}, valid_bleu: {valid_bleu:.4f} train_bleu: {train_bleu:.4f}')

  0%|          | 1/227 [00:22<1:25:27, 22.69s/it]


KeyboardInterrupt: ignored

In [ ]:
def translate_this_sentence(text: str):
    'translate the source sentence in string format into the target language'
    input = torch.tensor([[BOS] + tokenizers[SRC](text) + [EOS]]).to(DEVICE)
    output = translate(model, input)
    return decode_sentence(output[0], TRG)

translate_this_sentence("Eine Gruppe von Menschen steht vor einem Iglu.")